# Industry Screen Analytics Dashboard

Industry Fundamentals Analysis

## Thomson Reuters Business Classification List

https://tinyurl.com/y3kugwfs

## Inputs

In [2]:
classification = input("Select TRBC Hierarchy you would like to screen by (EconSector,BusinessSector, IndustryGroup, Industry, Activity):")
group = input("Please enter the {clas} code to screen for: ".format(clas=classification))
frequency = input("Fiscal Year or Fiscal Quarter results? (Select either FY or FQ)")
class_name = "TR.TRBC" + classification
classification_cd = 'TR.TRBC' + classification + "Code"

import refinitiv.dataplatform.eikon as ek
import pandas as pd
import numpy as np
import datetime as dt
import warnings
warnings.filterwarnings('ignore')
from dataquery import *
from IPython.display import display
import ipywidgets as widgets
import plotly.graph_objects as go

ek.set_app_key('DEFAULT_CODE_BOOK_APP_KEY')

SCREEN.universe = Equity(active=True,public=False,primary=True)
SCREEN.conditions = IN(classification_cd, group) ,IN('TR.ExchangeRegion','America') #, FORMULA("TR.CompanyMarketCap", ">= 100000000")
SCREEN.currency = "USD"

query = SCREEN.query

df,err = ek.get_data(query,[classification_cd,class_name,"TR.CompanyName"])
df["Company Name"] = np.where(df['Company Name'].isnull(),df['Instrument'],df['Company Name'])

industry_rics = df["Instrument"].to_list()
industry_rics = list(set(industry_rics))
industry_name = df.iloc[0,2]
x = dict(zip(df['Instrument'],df['Company Name']))

if frequency == 'FY':
    DataFields = ['TR.F.TotRevBizActiv.date','TR.F.TotRevBizActiv.fperiod','TR.F.TotRevBizActiv(Curn=USD)','TR.F.OpProfBefNonRecurIncExpn(Curn=USD)',
                  'TR.F.RnD(Curn=USD)','TR.F.RentalOpLeaseExpn(Curn=USD)','TR.F.ImpairTangIntangFixedAssets(Curn=USD)',
                  'TR.F.RestrChrg(Curn=USD)','TR.F.SaleOfTangIntangFixedAssetsGL(Curn=USD)','TR.F.NonRecurIncExpnTot(Curn=USD)',
                  'TR.F.CashSTInvst(Curn=USD)','TR.F.TotCap(Curn=USD)','TR.F.OpLeasePaymtDueInYr1(Curn=USD)',
                  'TR.F.OpLeasePaymtDueInYr2(Curn=USD)','TR.F.OpLeasePaymtDueInYr3(Curn=USD)','TR.F.OpLeasePaymtDueInYr4(Curn=USD)',
                  'TR.F.OpLeasePaymtDueInYr5(Curn=USD)','TR.F.OpLeasePaymtRemainMat(Curn=USD)','TR.F.GrossProfIndPropTot(Curn=USD)',
                  'TR.F.IncBefTax(Curn=USD)','TR.F.NetIncAfterTax(Curn=USD)','TR.F.OpExpnTot(Curn=USD)',
                  'TR.F.EBIT(Curn=USD)','TR.F.TotAssets(Curn=USD)',
                  'TR.F.WkgCap','TR.F.DebtTot(Curn=USD)','TR.F.TotShHoldEq(Curn=USD)',
                  'TR.F.DebtLTTot(Curn=USD)','TR.F.STDebtCurrPortOfLTDebt(Curn=USD)','TR.EV(Curn=USD)',
                  'TR.WACCcostofDebt','TR.WACC','TR.WACCTaxRate',
                  'TR.F.ZscoreManfWgts','TR.F.ZscoreNonManfWgts','TR.F.EarnRetentionRate',
                  'TR.F.ReturnAvgTotEqPct','TR.F.PriceToEPSBasicInclExOrdComTot','TR.F.AvgRcvblCollectDays',
                  'TR.F.InvntTurnover','TR.InventoryTurnoverAnnualized','TR.F.AvgInvntDays',
                  'TR.F.FixedAssetTurnover','TR.F.AssetTurnover','TR.F.CurrRatio','TR.F.QuickRatio','TR.F.AcctRcvblTurnover']
    
    data,err = ek.get_data(industry_rics,
                       DataFields,
                       parameters = {
                                     'SDate':'0',
                                     'EDate':'-5'
                                    })

elif frequency == "FQ":
    df1, err = ek.get_data(
                            instruments = industry_rics,
                            fields = ['TR.F.TotRevBizActiv.date','TR.F.TotRevBizActiv.fperiod',
                                      'TR.F.TotRevBizActiv(Curn=USD)','TR.F.OpProfBefNonRecurIncExpn(Curn=USD)','TR.F.RnD(Curn=USD)',
                                      'TR.F.RentalOpLeaseExpn(Curn=USD)','TR.F.ImpairTangIntangFixedAssets(Curn=USD)','TR.F.RestrChrg(Curn=USD)',
                                      'TR.F.SaleOfTangIntangFixedAssetsGL(Curn=USD)','TR.F.NonRecurIncExpnTot(Curn=USD)','TR.F.CashSTInvst(Curn=USD)',
                                      'TR.F.TotCap(Curn=USD)','TR.F.OpLeasePaymtDueInYr1(Curn=USD)','TR.F.OpLeasePaymtDueInYr2(Curn=USD)',
                                      'TR.F.OpLeasePaymtDueInYr3(Curn=USD)','TR.F.OpLeasePaymtDueInYr4(Curn=USD)','TR.F.OpLeasePaymtDueInYr5(Curn=USD)',
                                      'TR.F.OpLeasePaymtRemainMat(Curn=USD)','TR.F.GrossProfIndPropTot(Curn=USD)','TR.F.IncBefTax(Curn=USD)','TR.F.NetIncAfterTax(Curn=USD)',
                                      'TR.F.OpExpnTot(Curn=USD)','TR.F.EBIT','TR.F.TotAssets(Curn=USD)',
                                      'TR.F.WkgCap(Curn=USD)','TR.F.DebtTot(Curn=USD)','TR.F.TotShHoldEq(Curn=USD)',
                                      'TR.F.DebtLTTot(Curn=USD)','TR.F.STDebtCurrPortOfLTDebt(Curn=USD)','TR.EV(Curn=USD)','TR.WACCcostofDebt','TR.WACC',
                                      'TR.WACCTaxRate','TR.F.EarnRetentionRate','TR.InventoryTurnoverAnnualized','TR.F.CurrRatio','TR.F.QuickRatio'],
                            parameters = {
                                            'SDate':'0',
                                            'EDate':'-19',
                                            'Period':'FQ0',
                                            'Frq':'FQ'
                                          }
                            )

    df1['FY'] = df1['Financial Period Absolute'].str[0:6]
    df1 = df1[df1['Date']!=""]
    df1.dropna(subset=['Date', 'Financial Period Absolute'],inplace=True)

    df2, err = ek.get_data(
                            instruments = industry_rics,
                            fields = ['TR.F.ReturnAvgTotEqPct.fperiod','TR.F.ReturnAvgTotEqPct.date','TR.F.ReturnAvgTotEqPct','TR.F.ZscoreManfWgts',
                                      'TR.F.ZscoreNonManfWgts','TR.F.PriceToEPSBasicInclExOrdComTot','TR.F.AvgRcvblCollectDays',
                                      'TR.F.InvntTurnover','TR.F.AvgInvntDays','TR.F.FixedAssetTurnover','TR.F.AssetTurnover','TR.F.AcctRcvblTurnover'],
                            parameters = {
                                            'SDate':'0',
                                            'EDate':'-19',
                                            'Period':'FY0',
                                            'Frq':'FQ'
                                          }
                            )

    data = pd.merge(df1, df2,  how='left', left_on=['Instrument','FY'], right_on = ['Instrument','Financial Period Absolute'])
    data.drop(columns = ['Date_y','Financial Period Absolute_y'],inplace=True)
    data.drop_duplicates(inplace=True)
    data.rename(columns={'Date_x':'Date','Financial Period Absolute_x':'Financial Period Absolute'},inplace=True)

    

data["Date"] = data["Date"].str[:10]
data['Date'] = pd.to_datetime(data["Date"])
data['Company Name'] = data["Instrument"].map(x)
data['Tot Rev from Biz Activ'].replace(to_replace = 0,value = np.nan,inplace=True)

This_year = dt.date.today().year
Filing_years = [This_year - 1, This_year - 2, This_year - 3, This_year - 4, This_year - 5,This_year - 6]
Past5Data = data[(data["Date"].dt.year).isin(Filing_years)]
#Past5Data["R&D Expn"] = Past5Data["R&D Expn"].replace(to_replace = np.NaN,value=0)
Past5Data.reset_index(inplace=True,drop=True)
Past5Data.head(15)

Past5Data["Year"] = Past5Data["Date"].dt.year
Past5Data['Month'] = Past5Data['Date'].dt.month.apply(lambda x: 3 if x <= 3 else (6 if x <= 6 else(9 if x<=9 else 12)))
Past5Data["Day"] = Past5Data['Month'].apply(lambda x: 31 if x == 12 else(31 if x == 3 else(30 if x == 9 else 30)))
Past5Data[['Year','Month','Day']] = Past5Data[['Year','Month','Day']].astype(str)
Past5Data["DT"] = Past5Data['Year']+Past5Data['Month']+Past5Data['Day']
Past5Data["Date"] =pd.to_datetime(Past5Data["DT"],format = '%Y%m%d')
Past5Data.drop(columns=['Month',"Day","DT","Year"],inplace=True)
Past5Data["Year"] = Past5Data["Date"].dt.year

Past5Data['New Operating Income'] = Past5Data[['Op Profit Bef Non-Recur Inc/(Expn)','R&D Expn','Rental/Op Lease Expn']].sum(axis=1,skipna=True)

Past5Data['Tax'] = (Past5Data['WACC Tax Rate, (%)']/100)*(Past5Data['New Operating Income'])
Past5Data['Nopat'] = Past5Data['New Operating Income']-Past5Data['Tax']

Past5Data['TotOpLease'] = Past5Data[['Op Lease Paymt - Due in Yr 1','Op Lease Paymt - Due in Yr 2','Op Lease Paymt - Due in Yr 3','Op Lease Paymt - Due in Yr 4','Op Lease Paymt - Due in Yr 5','Op Lease Paymt - Remain Mat']].sum(axis=1,skipna=True)
Past5Data['PreTaxCoD'] = (Past5Data['WACC Cost of Debt, (%)']/100)/(1-(Past5Data['WACC Tax Rate, (%)']/100))

Past5Data['Total Adjusted Capital'] = Past5Data[['Tot Cap','TotOpLease','Impair - Tang & Intang Fixed Assets','Restr Chrg','Sale of Tang & Intang Fixed Assets - G/(L)','Non-Recur Inc/(Expn) - Tot','Cash & ST Invst']].sum(axis=1,skipna=True)
Past5Data['Total Adjusted Capital'].replace(to_replace = 0,value = np.nan,inplace=True)


Past5Data['Cost of Capital'] = (Past5Data['Weighted Average Cost of Capital, (%)']/100)*Past5Data['Total Adjusted Capital']


Past5Data['EVA'] = Past5Data['Nopat']-Past5Data['Cost of Capital']
Past5Data['EVA Margin'] = (Past5Data['EVA']/Past5Data['Tot Rev from Biz Activ'])*100

Past5Data['Pr_Pd'] = Past5Data['Date'].shift(-1)
Past5Data['Pr_Instr'] = Past5Data['Instrument'].shift(-1)
Past5Data['Total Revenue Prev Pd'] = Past5Data['Tot Rev from Biz Activ'].shift(-1)
Past5Data['EVA_prPd'] = Past5Data['EVA'].shift(-1)
Past5Data['Total Adjusted Capital Prev Pd'] = Past5Data['Total Adjusted Capital'].shift(-1)

Past5Data['Total Revenue Prev Pd'][(Past5Data['Date'] < Past5Data['Pr_Pd']) | (Past5Data['Instrument'] != Past5Data['Pr_Instr'])] = np.nan
Past5Data['EVA_prPd'][(Past5Data['Date'] < Past5Data['Pr_Pd']) | (Past5Data['Instrument'] != Past5Data['Pr_Instr'])] = np.nan
Past5Data['Total Adjusted Capital Prev Pd'][(Past5Data['Date'] < Past5Data['Pr_Pd']) | (Past5Data['Instrument'] != Past5Data['Pr_Instr'])] = np.nan

Past5Data['EVA Momentum Sales'] = 0
for index,row in Past5Data.iterrows():
    if row['Date']  > row['Pr_Pd']:
        Past5Data['EVA Momentum Sales'] = (Past5Data['EVA'] - Past5Data['EVA_prPd'])/(Past5Data['Total Revenue Prev Pd'])*100
    elif row['Date'] < row['Pr_Pd']:
        Past5Data['EVA Momentum Sales'] = np.nan
        
Past5Data['EVA Momentum Sales'][(Past5Data['Date'] < Past5Data['Pr_Pd']) | (Past5Data['Instrument'] != Past5Data['Pr_Instr'])] = np.nan

Past5Data['EVA Spread'] = Past5Data['EVA']/Past5Data['Total Adjusted Capital']

Past5Data['EVA Momentum vs Capital'] = 0 
for index,row in Past5Data.iterrows():
    if row['Date'] > row['Pr_Pd']:
        Past5Data['EVA Momentum vs Capital'] = (Past5Data['EVA'] - Past5Data['EVA_prPd'])/(Past5Data['Total Adjusted Capital Prev Pd']) * 100
    elif row['Date'] < row['Pr_Pd']:
        Past5Data['EVA Momentum vs Capital'] = np.nan
        
Past5Data['EVA Momentum vs Capital'][(Past5Data['Date'] < Past5Data['Pr_Pd']) | (Past5Data['Instrument'] != Past5Data['Pr_Instr'])] = np.nan

Past5Data['MVA'] = Past5Data['Enterprise Value (Daily Time Series)'] - Past5Data['Tot Cap']
Past5Data['EVA/MVA'] = Past5Data['EVA']/Past5Data['MVA']

Past5Data['EVA Momentum Sales']=Past5Data['EVA Momentum Sales'].replace([np.inf, -np.inf], np.nan)
Past5Data['EVA Momentum vs Capital']=Past5Data['EVA Momentum vs Capital'].replace([np.inf, -np.inf], np.nan)

Past5Data.head(15)

if frequency == "FY":
    Past5Data['Inventory Turnover Period Beg'] = Past5Data['Invnt Turnover'].shift(-1)
    Past5Data['Average Receivables Collection Days Period Beg'] = Past5Data['Avg Rcvbl Collect Days'].shift(-1)
    Past5Data['Inventory Turnover Period Beg'][Past5Data['Pr_Pd'].dt.year != Past5Data['Date'].dt.year -1] = np.nan
    Past5Data['Average Receivables Collection Days Period Beg'][Past5Data['Pr_Pd'].dt.year != Past5Data['Date'].dt.year -1] = np.nan
    Past5Data['Collection Period (Period Average)'] = Past5Data[['Average Receivables Collection Days Period Beg','Avg Rcvbl Collect Days']].mean(axis=1)
    Past5Data['Inventory Turns (Period Average)'] = Past5Data[['Inventory Turnover Period Beg','Invnt Turnover']].mean(axis=1)
elif frequency == "FQ":
    Past5Data['Pr_Pd2'] = Past5Data['Date'].shift(-4)
    Past5Data['Inventory Turnover Period Beg'] = Past5Data['Invnt Turnover'].shift(-4)
    Past5Data['Average Receivables Collection Days Period Beg'] = Past5Data['Avg Rcvbl Collect Days'].shift(-4)
    Past5Data['Inventory Turnover Period Beg'][Past5Data['Pr_Pd2'].dt.year != Past5Data['Date'].dt.year -1] = np.nan
    Past5Data['Average Receivables Collection Days Period Beg'][Past5Data['Pr_Pd2'].dt.year != Past5Data['Date'].dt.year-1] = np.nan
    Past5Data['Collection Period (Period Average)'] = Past5Data[['Average Receivables Collection Days Period Beg','Avg Rcvbl Collect Days']].mean(axis=1)
    Past5Data['Inventory Turns (Period Average)'] = Past5Data[['Inventory Turnover Period Beg','Invnt Turnover']].mean(axis=1)

Past5Data["Sustainable Growth Rate"] = (Past5Data['Return on Avg Tot Eq']/100)*(Past5Data['Earn Retention Rate']/100)*100

Past5Data['Gross Profit Margin on Sales'] = (Past5Data['Gross Profit - Ind/Prop - Tot']/Past5Data['Tot Rev from Biz Activ']) * 100
Past5Data['Gross Profit Margin on Sales']=Past5Data['Gross Profit Margin on Sales'].replace([np.inf, -np.inf], np.nan)

Past5Data['Net Profit Margin (Pretax)'] = (Past5Data['Inc Bef Taxes']/Past5Data['Tot Rev from Biz Activ'])*100
Past5Data['Net Profit Margin (After-tax)'] = (Past5Data['Net Inc After Taxes']/Past5Data['Tot Rev from Biz Activ'])*100
Past5Data['Net Profit Margin (Pretax)']=Past5Data['Net Profit Margin (Pretax)'].replace([np.inf, -np.inf], np.nan)
Past5Data['Net Profit Margin (After-tax)']=Past5Data['Net Profit Margin (After-tax)'].replace([np.inf, -np.inf], np.nan)

Past5Data['OpEx to Sales'] = (Past5Data['Op Expn - Tot']/Past5Data['Tot Rev from Biz Activ']) *100
Past5Data['OpEx to Sales'] = Past5Data['OpEx to Sales'].replace([np.inf, -np.inf], np.nan)

Past5Data['OpProf to Sales'] = (Past5Data['Op Profit Bef Non-Recur Inc/(Expn)']/Past5Data['Tot Rev from Biz Activ']) *100
Past5Data['OpProf to Sales'] = Past5Data['OpProf to Sales'].replace([np.inf, -np.inf], np.nan)

Past5Data['BEP'] = (Past5Data['EBIT']/Past5Data['Tot Assets'])*100
Past5Data['BEP'] = Past5Data['BEP'].replace([np.inf, -np.inf], np.nan)

Past5Data['ROA (AT)'] = (Past5Data['Net Inc After Taxes']/Past5Data['Tot Assets'])*100
Past5Data['ROA (AT)'] = Past5Data['ROA (AT)'].replace([np.inf, -np.inf], np.nan)

Past5Data['Gearing Ratio'] = Past5Data['Debt - Tot']/Past5Data['Tot Shareholders Eq']

Past5Data['Debt Ratio'] = Past5Data['Debt - Tot']/Past5Data['Tot Assets']

Past5Data['Debt to Equity'] = Past5Data['Debt - Tot']/Past5Data['Tot Shareholders Eq']

Past5Data['LT Debt to Equity'] = Past5Data['Debt - LT - Tot']/Past5Data['Tot Shareholders Eq']

Past5Data['Current Debt to Total Debt'] = Past5Data['ST Debt & Curr Port of LT Debt']/Past5Data['Debt - Tot']

Past5Data['Working Capital Turnover'] = abs(Past5Data['Tot Rev from Biz Activ'])/abs(Past5Data['Wkg Cap'])

Metrics_cols = ['Instrument','Company Name',"Date","Year",'Tot Rev from Biz Activ', 'Zscore - Manf Wgts', 'Zscore - Non - Manf Wgts',
                     'Sustainable Growth Rate','Price to EPS - Basic - Incl Exord Items Applic to Com - Tot','Gross Profit Margin on Sales', 'Net Profit Margin (Pretax)',
                     'Net Profit Margin (After-tax)', 'OpEx to Sales', 'OpProf to Sales', 'BEP', 'ROA (AT)','Return on Avg Tot Eq',
                     'Collection Period (Period Average)','Avg Rcvbl Collect Days',
                     'Inventory Turns (Period Average)','Invnt Turnover','Fixed Asset Turnover', 'Asset Turnover','Working Capital Turnover','Avg Invnt Days',
                     'Curr Ratio', 'Quick Ratio', 'Acct Rcvbl Turnover','Gearing Ratio', 'Debt Ratio', 'Debt to Equity','LT Debt to Equity', 'Current Debt to Total Debt',
                     'EVA', 'EVA Margin','EVA Momentum Sales', 'EVA Spread', 'EVA Momentum vs Capital', 'MVA','EVA/MVA']

Metrics_Col_Clean = ['Instrument','Company Name' ,"Date","Year",'Net Sales (USD)', 'Z-Score - Manufacturing Weights', 'Z-Score - NonManufacturing Weights',
                     'Sustainable Growth Rate (%)','Basic P/E Ratio (Incl Exord Items)','Gross Profit Margin on Sales (%)', 'Net Profit Margin (Pretax) (%)',
                     'Net Profit Margin (After-tax) (%)', 'Operating Expense to Sales (%)', 'Operating Profit to Sales (%)', 'Basic Earning Power (%)', 'Return on Assets (After Tax) (%)','Return on Avg Tot Equity (%)',
                     'Collection Period (Period Average) (Days)','Collection Period (Period End) (Days)',
                     'Inventory Turns (Period Average)','Inventory Turns (Period End)','Fixed Asset Turnover', 'Total Asset Turnover','Working Capital Turnover','Average Inventory Days',
                     'Current Ratio', 'Quick Ratio', 'Sales/Receivables','Gearing Ratio', 'Debt Ratio', 'Debt to Equity','LT Debt to Equity', 'Current Debt to Total Debt',
                     'EVA (USD)', 'EVA Margin (%)','EVA Momentum Sales (%)', 'EVA Spread', 'EVA Momentum vs Capital (%)', 'MVA (USD)','EVA/MVA']

three = ['Z-Score - Manufacturing Weights', 'Z-Score - NonManufacturing Weights','Basic P/E Ratio (Incl Exord Items)','Working Capital Turnover',
         'Inventory Turns (Period Average)','Inventory Turns (Period End)','Fixed Asset Turnover', 'Total Asset Turnover',
         'Current Ratio', 'Quick Ratio', 'Sales/Receivables','Gearing Ratio', 'Debt Ratio', 'Debt to Equity','LT Debt to Equity', 'Current Debt to Total Debt',
         'EVA Spread'  ,'EVA/MVA']

two = ['Gross Profit Margin on Sales (%)','Sustainable Growth Rate (%)','Net Profit Margin (Pretax) (%)','Net Profit Margin (After-tax) (%)',
       'Operating Expense to Sales (%)', 'Operating Profit to Sales (%)', 'Basic Earning Power (%)', 'Return on Assets (After Tax) (%)',
       'Return on Avg Tot Equity (%)','Collection Period (Period Average) (Days)','Collection Period (Period End) (Days)','Average Inventory Days',
       'EVA Margin (%)','EVA Momentum Sales (%)','EVA Momentum vs Capital (%)']

zero = ['Net Sales (USD)', 'EVA (USD)','MVA (USD)']


Metrics = Past5Data[Metrics_cols]
Metrics.columns = Metrics_Col_Clean
Metrics_RICS = Metrics["Company Name"].unique().tolist()
Metric_RICS = Metrics_RICS.sort()
Metrics_RICS = ['',classification] + Metrics_RICS
Columns = Metrics.columns.tolist()[4:]
Columns.sort()
Columns = [''] + Columns


if frequency == "FY":
    Metrics_Sum = Metrics.groupby(Metrics['Date'].dt.year).mean()
    Metrics_Sum.reset_index(inplace=True)
    Metrics_Sum["Instrument"] = classification
    Metrics_Sum["Company Name"] = classification
    Metrics_Sum['Date'] = Metrics_Sum['Date'].astype(str) + "1231"
    Metrics_Sum["Date"] = pd.to_datetime(Metrics_Sum["Date"],format = '%Y%m%d')
    Metrics_Sum = Metrics_Sum[Metrics_Col_Clean]
    Metrics_Sum = Metrics_Sum.iloc[::-1]
else:
    Metrics_Sum = Metrics.groupby(Metrics['Date']).mean()
    Metrics_Sum.reset_index(inplace=True)
    Metrics_Sum["Instrument"] = classification
    Metrics_Sum["Company Name"] = classification
    Metrics_Sum = Metrics_Sum[Metrics_Col_Clean]
    Metrics_Sum = Metrics_Sum.iloc[::-1]

Metrics = Metrics.append(Metrics_Sum)
Metrics = Metrics[Metrics["Year"] != Filing_years[-1]]
Metrics.reset_index(inplace=True,drop=True)


Metrics[three] = Metrics[three].round(3)
Metrics[two] = Metrics[two].round(2)
Metrics[zero] = Metrics[zero].round(0)


market_data = None
print("API Request Complete")

Select TRBC Hierarchy you would like to screen by (EconSector,BusinessSector, IndustryGroup, Industry, Activity): Industry
Please enter the Industry code to screen for:  52406010
Fiscal Year or Fiscal Quarter results? (Select either FY or FQ) FY


TypeError: <class 'pandas._libs.missing.NAType'> is not convertible to datetime

In [42]:
Metrics.to_csv("Industry_Met.csv")

#### Visualization Code

In [41]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

instrument_list = []
ind = Metrics[Metrics["Company Name"] == "Industry"]

if frequency == "FY":
    
    select_all = False
    
    cht = go.FigureWidget()
    cht.layout.template= 'plotly_dark'
    cht.add_trace(go.Scatter(mode='lines+markers+text',x=ind["Date"].dt.year, y=ind['EVA (USD)'], name="Industry", connectgaps=True,
                             text = ind['EVA (USD)'],textposition = 'bottom left',texttemplate="$%{text:,}"))
    cht.update_layout(
            autosize=True,
            margin=dict(
                t=30, b=15, l=30, r=45
            ),
        showlegend=True,
        title_text = industry_name + " - EVA (USD)",
        title_font_size = 15,
        width=1500,
        height=500,
        xaxis=dict(title_text = "Fiscal Year",
                   title_font_size = 12,
                   tickmode = 'array',
                   tickvals = [2015,2016,2017,2018,2019]),
        yaxis=dict(title_text = '$ (USD)',
                   title_font_size = 12),
        legend=dict(orientation="h",
                    yanchor="top",
                    y=1.02,
                    xanchor="right",
                    x=1
                    )

    )

    cht1 = go.FigureWidget()
    cht1.layout.template= 'plotly_dark'
    cht1.add_trace(go.Bar(x=ind["Date"].dt.year, y=ind['EVA (USD)'],name ="Industry",text = ind['EVA (USD)'],textposition = 'auto',texttemplate="$%{text:,}",))
    cht1.update_layout(
            autosize=True,
            margin=dict(
                t=30, b=15, l=30, r=45
            ),
        legend=dict(orientation="h",
                    yanchor="top",
                    y=1.02,
                    xanchor="right",
                    x=1
                    ),
        showlegend=True,
        title_text = industry_name + " - EVA (USD)",
        title_font_size = 15,
        width=750,
        height=500,
        xaxis=dict(title_text = "Fiscal Year",
                   title_font_size = 12,
                   tickmode = 'array',
                   tickvals = [2015,2016,2017,2018,2019]),
        yaxis=dict(title_text = '$ (USD)',
                   title_font_size = 12),
        uniformtext_minsize=6



    )

    cht2 = go.FigureWidget()
    cht2.layout.template= 'plotly_dark'
    cht2.add_trace(go.Bar(x=ind["Date"].dt.year, y=ind['EVA (USD)'],name ="Industry",text = ind['EVA (USD)'],textposition = 'auto',texttemplate="$%{text:,}",))
    cht2.update_layout(
            autosize=True,
            margin=dict(
                t=30, b=15, l=30, r=45
            ),
        legend=dict(orientation="h",
                    yanchor="top",
                    y=1.02,
                    xanchor="right",
                    x=1
                    ),
        showlegend=True,
        title_text = industry_name + " - EVA (USD)",
        title_font_size = 15,
        width=750,
        height=500,
        xaxis=dict(title_text = "Fiscal Year",
                   title_font_size = 12,
                   tickmode = 'array',
                   tickvals = [2015,2016,2017,2018,2019]),
        yaxis=dict(title_text = '$ (USD)',
                   title_font_size = 12),
        uniformtext_minsize=6

    )

    cht3 = go.FigureWidget()
    cht3.layout.template= 'plotly_dark'
    cht3.add_trace(go.Bar(x=ind["Date"].dt.year, y=ind['EVA (USD)'],name ="Industry",text = ind['EVA (USD)'],textposition = 'auto',texttemplate="$%{text:,}"))
    cht3.update_layout(
            autosize=True,
            margin=dict(
                t=30, b=15, l=30, r=45
            ),
        legend=dict(orientation="h",
                    yanchor="top",
                    y=1.02,
                    xanchor="right",
                    x=1
                    ),
        showlegend=True,
        title_text = industry_name + " - EVA (USD)",
        title_font_size = 15,
        width=750,
        height=500,
        xaxis=dict(title_text = "Fiscal Year",
                   title_font_size = 12,
                   tickmode = 'array',
                   tickvals = [2015,2016,2017,2018,2019]),
        yaxis=dict(title_text = '$ (USD)',
                   title_font_size = 12),
        uniformtext_minsize=6
    )

    cht4 = go.FigureWidget()
    cht4.layout.template= 'plotly_dark'
    cht4.add_trace(go.Scatter(mode='lines+markers+text',x=ind["Date"].dt.year, y=ind['EVA (USD)'], name="Industry", connectgaps=True,
                              text = ind['EVA (USD)'],textposition = 'bottom left',texttemplate="$%{text:,}"))
    cht4.update_layout(
            autosize=True,
            margin=dict(
                t=30, b=15, l=30, r=45
            ),
        legend=dict(orientation="h",
                    yanchor="top",
                    y=1.02,
                    xanchor="right",
                    x=1
                    ),
        showlegend=True,
        title_text = industry_name + " - EVA (USD)",
        title_font_size = 15,
        width=750,
        height=500,
        xaxis=dict(title_text = "Fiscal Year",
                   title_font_size = 12,
                   tickmode = 'array',
                   tickvals = [2015,2016,2017,2018,2019]),
        yaxis=dict(title_text = '$ (USD)',
                   title_font_size = 12)
    )

    cht5 = go.FigureWidget()
    cht5.layout.template= 'plotly_dark'
    cht5.add_trace(go.Bar(x=ind["Date"].dt.year, y=ind['EVA (USD)'],name ="Industry",text = ind['EVA (USD)'],textposition = 'auto',texttemplate="$%{text:,}"))
    cht5.update_layout(
            autosize=True,
            margin=dict(
                t=30, b=15, l=30, r=45
            ),
        legend=dict(orientation="h",
                    yanchor="top",
                    y=1.02,
                    xanchor="right",
                    x=1
                    ),
        showlegend=True,
        title_text = industry_name + " - EVA (USD)",
        title_font_size = 15,
        width=1500,
        height=500,
        xaxis=dict(title_text = "Fiscal Year",
                   title_font_size = 12,
                   tickmode = 'array',
                   tickvals = [2015,2016,2017,2018,2019]),
        yaxis=dict(title_text = '$ (USD)',
                   title_font_size = 12),
        uniformtext_minsize=6
    )
    charts = [cht,cht1,cht2,cht3,cht4,cht5]
    
    instrument_list.append("Industry")    
    
    def add_instrument(value):
        global instrument_list, cht,cht1,cht2,cht3,cht4,cht5, market_data, Single_Instrument, ColumnPicker
        ric = value['new']
        if (ric not in instrument_list) and (ric != ''):
            instrument_list.append(ric)
            df = Metrics[Metrics['Company Name'].isin(instrument_list)]
            column = 'EVA (USD)'
            for ric in instrument_list:
                y = df[df["Company Name"] == ric][["Date",column]]
                traces = [go.Scatter(mode='lines+markers+text',x=y["Date"].dt.year,y=y[column],name=ric,connectgaps=True,text = y[column],textposition = 'bottom left',texttemplate="$%{text:,}"),
                          go.Bar(x=y["Date"].dt.year, y=y[column],name =ric,text = y[column],textposition = 'auto',texttemplate="$%{text:,}"),
                          go.Bar(x=y["Date"].dt.year, y=y[column],name =ric,text = y[column],textposition = 'auto',texttemplate="$%{text:,}"),
                          go.Bar(x=y["Date"].dt.year, y=y[column],name =ric,text = y[column],textposition = 'auto',texttemplate="$%{text:,}"),
                          go.Scatter(mode='lines+markers+text',x=y["Date"].dt.year,y=y[column],name=ric,connectgaps=True,text = y[column],textposition = 'bottom left',texttemplate="$%{text:,}"),
                          go.Bar(x=y["Date"].dt.year, y=y[column],name =ric,text = y[column],textposition = 'auto',texttemplate="$%{text:,}")]
            for i in range(0,6):
                charts[i].add_trace(traces[i])

            market_data = df

        return

    def clear_chart(value):
        global cht,cht1,cht2,cht3, market_data, instrument_list
        market_data = pd.DataFrame()
        for c in charts:
            c.data=[]
        instrument_list = []
        return
        

    def select_metric(value):
        global ColumnPicker, cht, market_data
        df = market_data
        metric = value['new']
        if metric in ['Net Sales (USD)', 'EVA (USD)','MVA (USD)']:
            template = "$%{text:,}"
        else:
            template = "%{text}"
        if metric == '':
            return
        else:
            cht.data = []
            for ric in instrument_list:
                y = df[df["Company Name"] == ric][["Date",metric]]
                text = y[metric]
                trace= go.Scatter(
                                   mode='lines+markers+text',
                                   x=y["Date"].dt.year,
                                   y=y[metric],
                                   name=ric,
                                   connectgaps=True,
                                   text = text,
                                   textposition = 'bottom left',
                                   texttemplate= template
                                 )

                cht.add_trace(trace)
                cht.update_layout(title_text = "{ind} - {met}".format(ind = industry_name,met=metric),
                                  title_font_size = 15,
                                  yaxis=dict(title_text = metric,
                                             title_font_size = 12))

        return


    def select_metric1(value):
        global ColumnPicker1, cht1, market_data
        df = market_data
        metric = value['new']
        if metric in ['Net Sales (USD)', 'EVA (USD)','MVA (USD)']:
            template = "$%{text:,}"
        else:
            template = "%{text}"
        if metric == '':
            return
        else:
            cht1.data = []
            for ric in instrument_list:
                y = df[df["Company Name"] == ric][["Date",metric]]
                trace1 = go.Bar(
                        x=y["Date"].dt.year,
                        y=y[metric], 
                        name=ric,
                        text = y[metric],
                        textposition = 'auto',
                        texttemplate=template

                    )
                cht1.add_trace(trace1)
                cht1.update_layout(title_text = "{ind} - {met}".format(ind = industry_name,met=metric),
                                    title_font_size = 15,
                                    yaxis=dict(title_text = metric,
                                               title_font_size = 12))

        return



    def select_metric2(value):
        global ColumnPicker2, cht2, market_data
        df = market_data
        metric = value['new']
        if metric in ['Net Sales (USD)', 'EVA (USD)','MVA (USD)']:
            template = "$%{text:,}"
        else:
            template = "%{text}"
        if metric == '':
            return
        else:
            cht2.data = []
            for ric in instrument_list:
                y = df[df["Company Name"] == ric][["Date",metric]]
                trace2 = go.Bar(
                        x=y["Date"].dt.year,
                        y=y[metric], 
                        name=ric,
                        text = y[metric],
                        textposition = 'auto',
                        texttemplate=template
                    )
                cht2.add_trace(trace2)
                cht2.update_layout(title_text = "{ind} - {met}".format(ind = industry_name,met=metric),
                                  title_font_size = 15,
                                  yaxis=dict(title_text = metric,
                                             title_font_size = 12))

        return



    def select_metric3(value):
        global ColumnPicker3, cht3, market_data
        df = market_data
        metric = value['new']
        if metric in ['Net Sales (USD)', 'EVA (USD)','MVA (USD)']:
            template = "$%{text:,}"
        else:
            template = "%{text}"
        if metric == '':
            return
        else:    
            cht3.data = []
            for ric in instrument_list:
                y = df[df["Company Name"] == ric][["Date",metric]]
                trace3 = go.Bar(
                        x=y["Date"].dt.year,
                        y=y[metric], 
                        name=ric,
                        text = y[metric],
                        textposition = 'auto',
                        texttemplate=template
                    )
                cht3.add_trace(trace3)
                cht3.update_layout(title_text = "{ind} - {met}".format(ind = industry_name,met=metric),
                                  title_font_size = 15,
                                  yaxis=dict(title_text = metric,
                                             title_font_size = 12))

        return

    def select_metric4(value):
        global ColumnPicker4, cht4, market_data
        df = market_data
        metric = value['new']
        if metric in ['Net Sales (USD)', 'EVA (USD)','MVA (USD)']:
            template = "$%{text:,}"
        else:
            template = "%{text}"
        if metric == '':
            return
        else:
            cht4.data = []   
            for ric in instrument_list:
                y = df[df["Company Name"] == ric][["Date",metric]]
                trace4 = go.Scatter(
                        mode='lines+markers+text', 
                        x=y["Date"].dt.year,
                        y=y[metric], 
                        name=ric,
                        connectgaps=True,
                        text = y[metric],
                        textposition = 'bottom left',
                        texttemplate= template
                    )
                cht4.add_trace(trace4)
                cht4.update_layout(title_text = "{ind} - {met}".format(ind = industry_name,met=metric),
                                  title_font_size = 15,
                                  yaxis=dict(title_text = metric,
                                             title_font_size = 12))

        return

    def select_metric5(value):
        global ColumnPicker5, cht5, market_data
        df = market_data
        metric = value['new']
        if metric in ['Net Sales (USD)', 'EVA (USD)','MVA (USD)']:
            template = "$%{text:,}"
        else:
            template = "%{text}"
        if metric == '':
            return
        else:
            cht5.data = []    
            for ric in instrument_list:
                y = df[df["Company Name"] == ric][["Date",metric]]
                trace5 = go.Bar(
                        x=y["Date"].dt.year,
                        y=y[metric], 
                        name=ric,
                        text = y[metric],
                        textposition = 'auto',
                        texttemplate=template
                    )
                cht5.add_trace(trace5)
                cht5.update_layout(title_text = "{ind} - {met}".format(ind = industry_name,met=metric),
                                  title_font_size = 15,
                                  yaxis=dict(title_text = metric,
                                             title_font_size = 12))

        return



    button_clear = widgets.Button(description='Clear data')
    button_clear.on_click(clear_chart)
    



    Single_Instrument = widgets.Dropdown(
                                            options=Metrics_RICS,
                                            value=Metrics_RICS[0],
                                            description='Instrument:',
                                            disabled=False,
                                        )
    Single_Instrument.observe(add_instrument, names = 'value')

    ColumnPicker = widgets.Dropdown(
        options=Columns,
        value=Columns[0],
        #rows=10,
        description='Metric',
        disabled=False
    )
    ColumnPicker.observe(select_metric,names = 'value')


    ColumnPicker1 = widgets.Dropdown(
        options=Columns,
        value=Columns[0],
        #rows=10,
        description='Metric',
        disabled=False
    )
    ColumnPicker1.observe(select_metric1,names = 'value')



    ColumnPicker2 = widgets.Dropdown(
        options=Columns,
        value=Columns[0],
        #rows=10,
        description='Metric',
        disabled=False
    )
    ColumnPicker2.observe(select_metric2,names = 'value')

    ColumnPicker3 = widgets.Dropdown(
        options=Columns,
        value=Columns[0],
        #rows=10,
        description='Metric',
        disabled=False
    )
    ColumnPicker3.observe(select_metric3,names = 'value')

    ColumnPicker4 = widgets.Dropdown(
        options=Columns,
        value=Columns[0],
        #rows=10,
        description='Metric',
        disabled=False
    )
    ColumnPicker4.observe(select_metric4,names = 'value')

    ColumnPicker5 = widgets.Dropdown(
        options=Columns,
        value=Columns[0],
        #rows=10,
        description='Metric',
        disabled=False
    )
    ColumnPicker5.observe(select_metric5,names = 'value')


    header11 = widgets.HBox([ColumnPicker,button_clear])
    box11 = widgets.VBox([header11, cht])

    header12 = widgets.HBox([ColumnPicker1,button_clear])
    box12 = widgets.VBox([header12, cht1])

    header13 = widgets.HBox([ColumnPicker2,button_clear])
    box13 = widgets.VBox([header13, cht2])

    box1 = widgets.HBox([box12,box13])

    header21 = widgets.HBox([ColumnPicker3,button_clear])
    box21 = widgets.VBox([header21, cht3])

    header22 = widgets.HBox([ColumnPicker4,button_clear])
    box22 = widgets.VBox([header22, cht4])

    header23 = widgets.HBox([ColumnPicker5,button_clear])
    box23 = widgets.VBox([header23, cht5])

    box2 = widgets.HBox([box21,box22])

    box = widgets.VBox([Single_Instrument,box11,box1,box2,box23])

    display(box)

elif frequency == "FQ":
    cht = go.FigureWidget()
    cht.layout.template= 'plotly_dark'
    cht.add_trace(go.Scatter(mode='lines+markers+text',x=ind["Date"], y=ind['EVA (USD)'], name="Industry", connectgaps=True,
                             ))
    cht.update_layout(
            autosize=True,
            margin=dict(
                t=30, b=15, l=30, r=45
            ),
        showlegend=True,
        title_text = industry_name + " - EVA (USD)",
        title_font_size = 15,
        width=1500,
        height=500,
        xaxis=dict(title_text = "Fiscal Year",
                   title_font_size = 12,
                   ),
        yaxis=dict(title_text = '$ (USD)',
                   title_font_size = 12),
        legend=dict(orientation="h",
                    yanchor="top",
                    y=1.02,
                    xanchor="right",
                    x=1
                    )

    )

    cht1 = go.FigureWidget()
    cht1.layout.template= 'plotly_dark'
    cht1.add_trace(go.Bar(x=ind["Date"], y=ind['EVA (USD)'],name ="Industry"))
    cht1.update_layout(
            autosize=True,
            margin=dict(
                t=30, b=15, l=30, r=45
            ),
        legend=dict(orientation="h",
                    yanchor="top",
                    y=1.02,
                    xanchor="right",
                    x=1
                    ),
        showlegend=True,
        title_text = industry_name + " - EVA (USD)",
        title_font_size = 15,
        width=750,
        height=500,
        xaxis=dict(title_text = "Fiscal Year",
                   title_font_size = 12,
                   ),
        yaxis=dict(title_text = '$ (USD)',
                   title_font_size = 12),
        uniformtext_minsize=6



    )

    cht2 = go.FigureWidget()
    cht2.layout.template= 'plotly_dark'
    cht2.add_trace(go.Bar(x=ind["Date"], y=ind['EVA (USD)'],name ="Industry"))
    cht2.update_layout(
            autosize=True,
            margin=dict(
                t=30, b=15, l=30, r=45
            ),
        legend=dict(orientation="h",
                    yanchor="top",
                    y=1.02,
                    xanchor="right",
                    x=1
                    ),
        showlegend=True,
        title_text = industry_name + " - EVA (USD)",
        title_font_size = 15,
        width=750,
        height=500,
        xaxis=dict(title_text = "Fiscal Year",
                   title_font_size = 12,
                   ),
        yaxis=dict(title_text = '$ (USD)',
                   title_font_size = 12),
        uniformtext_minsize=12

    )

    cht3 = go.FigureWidget()
    cht3.layout.template= 'plotly_dark'
    cht3.add_trace(go.Bar(x=ind["Date"], y=ind['EVA (USD)'],name ="Industry"))
    cht3.update_layout(
            autosize=True,
            margin=dict(
                t=30, b=15, l=30, r=45
            ),
        legend=dict(orientation="h",
                    yanchor="top",
                    y=1.02,
                    xanchor="right",
                    x=1
                    ),
        showlegend=True,
        title_text = industry_name + " - EVA (USD)",
        title_font_size = 15,
        width=750,
        height=500,
        xaxis=dict(title_text = "Fiscal Year",
                   title_font_size = 12,
                   ),
        yaxis=dict(title_text = '$ (USD)',
                   title_font_size = 12),
        uniformtext_minsize=6
    )

    cht4 = go.FigureWidget()
    cht4.layout.template= 'plotly_dark'
    cht4.add_trace(go.Scatter(mode='lines+markers+text',x=ind["Date"], y=ind['EVA (USD)'], name="Industry", connectgaps=True,
                              ))
    cht4.update_layout(
            autosize=True,
            margin=dict(
                t=30, b=15, l=30, r=45
            ),
        legend=dict(orientation="h",
                    yanchor="top",
                    y=1.02,
                    xanchor="right",
                    x=1
                    ),
        showlegend=True,
        title_text = industry_name + " - EVA (USD)",
        title_font_size = 15,
        width=750,
        height=500,
        xaxis=dict(title_text = "Fiscal Year",
                   title_font_size = 12,
                   ),
        yaxis=dict(title_text = '$ (USD)',
                   title_font_size = 12)
    )

    cht5 = go.FigureWidget()
    cht5.layout.template= 'plotly_dark'
    cht5.add_trace(go.Bar(x=ind["Date"], y=ind['EVA (USD)'],name ="Industry"))
    cht5.update_layout(
            autosize=True,
            margin=dict(
                t=30, b=15, l=30, r=45
            ),
        legend=dict(orientation="h",
                    yanchor="top",
                    y=1.02,
                    xanchor="right",
                    x=1
                    ),
        showlegend=True,
        title_text = industry_name + " - EVA (USD)",
        title_font_size = 15,
        width=1500,
        height=500,
        xaxis=dict(title_text = "Fiscal Year",
                   title_font_size = 12,
                   ),
        yaxis=dict(title_text = '$ (USD)',
                   title_font_size = 12),
        uniformtext_minsize=6
    )


    charts = [cht,cht1,cht2,cht3,cht4,cht5]




    instrument_list.append("Industry")    

    def add_instrument(value):
        global instrument_list, cht, market_data, Single_Instrument, ColumnPicker
        ric = value['new']
        if (ric not in instrument_list) and (ric != ''):
            instrument_list.append(ric)
            df = Metrics[Metrics['Company Name'].isin(instrument_list)]
            column = 'EVA (USD)'
            for ric in instrument_list:
                y = df[df["Company Name"] == ric][["Date",column]]
                traces = [go.Scatter(mode='lines+markers+text',x=y["Date"],y=y[column],name=ric,connectgaps=True),
                          go.Bar(x=y["Date"], y=y[column],name =ric),
                          go.Bar(x=y["Date"], y=y[column],name =ric),
                          go.Bar(x=y["Date"], y=y[column],name =ric),
                          go.Scatter(mode='lines+markers+text',x=y["Date"],y=y[column],name=ric,connectgaps=True),
                          go.Bar(x=y["Date"], y=y[column],name =ric)]
            for i in range(0,6):
                charts[i].add_trace(traces[i])

            market_data = df

        return

    def clear_chart(value):
        global cht,cht1,cht2,cht3, market_data, instrument_list
        market_data = pd.DataFrame()
        for c in charts:
            c.data=[]
        instrument_list = []
        return

    def select_metric(value):
        global ColumnPicker, cht, market_data
        df = market_data
        metric = value['new']
        if metric in ['Net Sales (USD)', 'EVA (USD)','MVA (USD)']:
            template = "$%{text:,}"
        else:
            template = "%{text}"
        if metric == '':
            return
        else:
            cht.data = []
            for ric in instrument_list:
                y = df[df["Company Name"] == ric][["Date",metric]]
                trace= go.Scatter(
                                   mode='lines+markers+text',
                                   x=y["Date"],
                                   y=y[metric],
                                   name=ric,
                                   connectgaps=True,
                                   
                                 )

                cht.add_trace(trace)
                cht.update_layout(title_text = "{ind} - {met}".format(ind = industry_name,met=metric),
                                  title_font_size = 15,
                                  yaxis=dict(title_text = metric,
                                             title_font_size = 12))

        return


    def select_metric1(value):
        global ColumnPicker1, cht1, market_data
        df = market_data
        metric = value['new']
        if metric in ['Net Sales (USD)', 'EVA (USD)','MVA (USD)']:
            template = "$%{text:,}"
        else:
            template = "%{text}"
        if metric == '':
            return
        else:
            cht1.data = []
            for ric in instrument_list:
                y = df[df["Company Name"] == ric][["Date",metric]]
                trace1 = go.Bar(
                        x=y["Date"],
                        y=y[metric], 
                        name=ric

                    )
                cht1.add_trace(trace1)
                cht1.update_layout(title_text = "{ind} - {met}".format(ind = industry_name,met=metric),
                                    title_font_size = 15,
                                    yaxis=dict(title_text = metric,
                                               title_font_size = 12))

        return



    def select_metric2(value):
        global ColumnPicker2, cht2, market_data
        df = market_data
        metric = value['new']
        if metric in ['Net Sales (USD)', 'EVA (USD)','MVA (USD)']:
            template = "$%{text:,}"
        else:
            template = "%{text}"
        if metric == '':
            return
        else:
            cht2.data = []
            for ric in instrument_list:
                y = df[df["Company Name"] == ric][["Date",metric]]
                trace2 = go.Bar(
                        x=y["Date"],
                        y=y[metric], 
                        name=ric,

                    )
                cht2.add_trace(trace2)
                cht2.update_layout(title_text = "{ind} - {met}".format(ind = industry_name,met=metric),
                                  title_font_size = 15,
                                  yaxis=dict(title_text = metric,
                                             title_font_size = 12))

        return



    def select_metric3(value):
        global ColumnPicker3, cht3, market_data
        df = market_data
        metric = value['new']
        if metric in ['Net Sales (USD)', 'EVA (USD)','MVA (USD)']:
            template = "$%{text:,}"
        else:
            template = "%{text}"
        if metric == '':
            return
        else:    
            cht3.data = []
            for ric in instrument_list:
                y = df[df["Company Name"] == ric][["Date",metric]]
                trace3 = go.Bar(
                        x=y["Date"],
                        y=y[metric], 
                        name=ric,

                    )
                cht3.add_trace(trace3)
                cht3.update_layout(title_text = "{ind} - {met}".format(ind = industry_name,met=metric),
                                  title_font_size = 15,
                                  yaxis=dict(title_text = metric,
                                             title_font_size = 12))

        return

    def select_metric4(value):
        global ColumnPicker4, cht4, market_data
        df = market_data
        metric = value['new']
        if metric in ['Net Sales (USD)', 'EVA (USD)','MVA (USD)']:
            template = "$%{text:,}"
        else:
            template = "%{text}"
        if metric == '':
            return
        else:
            cht4.data = []   
            for ric in instrument_list:
                y = df[df["Company Name"] == ric][["Date",metric]]
                trace4 = go.Scatter(
                        mode='lines+markers+text', 
                        x=y["Date"],
                        y=y[metric], 
                        name=ric,
                        connectgaps=True,
                        
                    )
                cht4.add_trace(trace4)
                cht4.update_layout(title_text = "{ind} - {met}".format(ind = industry_name,met=metric),
                                  title_font_size = 15,
                                  yaxis=dict(title_text = metric,
                                             title_font_size = 12))

        return

    def select_metric5(value):
        global ColumnPicker5, cht5, market_data
        df = market_data
        metric = value['new']
        if metric in ['Net Sales (USD)', 'EVA (USD)','MVA (USD)']:
            template = "$%{text:,}"
        else:
            template = "%{text}"
        if metric == '':
            return
        else:
            cht5.data = []    
            for ric in instrument_list:
                y = df[df["Company Name"] == ric][["Date",metric]]
                trace5 = go.Bar(
                        x=y["Date"],
                        y=y[metric], 
                        name=ric,
                       
                    )
                cht5.add_trace(trace5)
                cht5.update_layout(title_text = "{ind} - {met}".format(ind = industry_name,met=metric),
                                  title_font_size = 15,
                                  yaxis=dict(title_text = metric,
                                             title_font_size = 12))

        return



    button_clear = widgets.Button(description='Clear data')
    button_clear.on_click(clear_chart)


    Single_Instrument = widgets.Dropdown(
                                            options=Metrics_RICS,
                                            value=Metrics_RICS[0],
                                            description='Instrument:',
                                            disabled=False,
                                        )
    Single_Instrument.observe(add_instrument, names = 'value')

    ColumnPicker = widgets.Dropdown(
        options=Columns,
        value=Columns[0],
        #rows=10,
        description='Metric',
        disabled=False
    )
    ColumnPicker.observe(select_metric,names = 'value')


    ColumnPicker1 = widgets.Dropdown(
        options=Columns,
        value=Columns[0],
        #rows=10,
        description='Metric',
        disabled=False
    )
    ColumnPicker1.observe(select_metric1,names = 'value')



    ColumnPicker2 = widgets.Dropdown(
        options=Columns,
        value=Columns[0],
        #rows=10,
        description='Metric',
        disabled=False
    )
    ColumnPicker2.observe(select_metric2,names = 'value')

    ColumnPicker3 = widgets.Dropdown(
        options=Columns,
        value=Columns[0],
        #rows=10,
        description='Metric',
        disabled=False
    )
    ColumnPicker3.observe(select_metric3,names = 'value')

    ColumnPicker4 = widgets.Dropdown(
        options=Columns,
        value=Columns[0],
        #rows=10,
        description='Metric',
        disabled=False
    )
    ColumnPicker4.observe(select_metric4,names = 'value')

    ColumnPicker5 = widgets.Dropdown(
        options=Columns,
        value=Columns[0],
        #rows=10,
        description='Metric',
        disabled=False
    )
    ColumnPicker5.observe(select_metric5,names = 'value')


    header11 = widgets.HBox([ColumnPicker,button_clear])
    box11 = widgets.VBox([header11, cht])

    header12 = widgets.HBox([ColumnPicker1,button_clear])
    box12 = widgets.VBox([header12, cht1])

    header13 = widgets.HBox([ColumnPicker2,button_clear])
    box13 = widgets.VBox([header13, cht2])

    box1 = widgets.HBox([box12,box13])

    header21 = widgets.HBox([ColumnPicker3,button_clear])
    box21 = widgets.VBox([header21, cht3])

    header22 = widgets.HBox([ColumnPicker4,button_clear])
    box22 = widgets.VBox([header22, cht4])

    header23 = widgets.HBox([ColumnPicker5,button_clear])
    box23 = widgets.VBox([header23, cht5])

    box2 = widgets.HBox([box21,box22])

    box = widgets.VBox([Single_Instrument,box11,box1,box2,box23])

    display(box)